In [ ]:
from utils import *
import tensorflow as tf
import os, cv2
import numpy as np
from PIL import Image

In [ ]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
temp_dir = "./temp/"

In [ ]:
if bool(1-os.path.exists(temp_dir)):
    os.mkdir(temp_dir)
    print("temp_dir not exists, create one.")
getFrame("../data/demo.mp4", temp_dir, 1)
print("getFrame finished.")
blurryList(temp_dir,temp_dir,2)
print("get LR pics finished.")

In [ ]:
images_to_video(temp_dir,"./","LR_demo",20)

In [ ]:
def customized_loss(y_true, y_pred):
    return tf.reduce_sum(tf.reduce_mean(tf.square(y_pred - y_true)))
model = tf.keras.models.load_model("./model/SRCNN.h5",custom_objects={"customized_loss":customized_loss})

In [ ]:
lr_data = load_images(temp_dir)
print(lr_data.shape)

In [ ]:
for i in range(len(lr_data)):
    hr_img = model.predict(BiCubicList(lr_data[i:i+1],4))[0]
    img = Image.fromarray(np.uint8(hr_img),"RGB")
    img.save(os.path.join(temp_dir,"%d.jpg"%(i+1)))
    # print("save %d" % i)

In [ ]:
images_to_video(temp_dir,"./","HR_by_Net",20)

In [ ]:
import shutil
shutil.rmtree(temp_dir)